In [2]:
import pandas as pd
import numpy as np
import mysql
from mysql import connector

### 1. csv 파일을 가져와 DB에 넣기 위해 데이터 가공

* 연령별 전국민 수 통계 데이터

In [6]:
all = pd.read_csv("../eda_data/연령_장래.csv")
all = all.replace('-', 0).fillna(0)
pop_all = all.drop("가정별", axis=1)
pop_all = pop_all.drop("성별", axis=1)
pop_all.rename(columns={'시점': '년도'}, inplace=True)
pop_all['데이터'].astype(int)
pop_all

,년도,연령별,데이터
0,1960,0 - 4세,4590812
1,1960,5 - 9세,3350533
2,1960,10 - 14세,2646238
3,1960,15 - 19세,2418444
4,1960,20 - 24세,2322705
...,...,...,...
165,2024,60 - 64세,4211091
166,2024,65 - 69세,3509250
167,2024,70 - 74세,2322783
168,2024,75 - 79세,1723662


In [8]:
def age_group(row):
    age_str = row['연령별']  # 연령 범위 문자열 추출
    if age_str == '80세이상':
        return '6.노년'
    else:
        age = age_str.split(' - ')[0]  # 연령 범위의 시작 연령 추출
        if age.isdigit():  # 숫자인지 확인
            age = int(age)  # 정수로 변환
            if age < 5:
                return '0.영유아'
            elif age < 12:
                return '1.아동'
            elif age < 18:
                return '2.청소년'
            elif age < 29:
                return '3.청년'
            elif age < 49:
                return '4.중년'
            elif age < 65:
                return '5.장년'
            else:
                return '6.노년'
        else:
            return '6.노년'  # 숫자가 아닌 경우(예외 처리)

# 데이터프레임에 함수 적용
pop_all['연령별 범위'] = pop_all.apply(age_group, axis=1)

pop_all = pop_all[['년도', '연령별', '연령별 범위', '데이터']]
pop_all


,년도,연령별,연령별 범위,데이터
0,1960,0 - 4세,0.영유아,4590812
1,1960,5 - 9세,1.아동,3350533
2,1960,10 - 14세,1.아동,2646238
3,1960,15 - 19세,2.청소년,2418444
4,1960,20 - 24세,3.청년,2322705
...,...,...,...,...
165,2024,60 - 64세,5.장년,4211091
166,2024,65 - 69세,6.노년,3509250
167,2024,70 - 74세,6.노년,2322783
168,2024,75 - 79세,6.노년,1723662


* 년도별 고령화비율 통계 데이터

In [5]:
pop_old = pd.read_csv("../data2/고령화비율.csv")
old = pop_old.drop([0], inplace=True)
old = pop_old.replace('-', 0).fillna(0)
old.reset_index(drop=True, inplace=True)
old

,행정구역별(1),2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,전국,7.0,7.4,7.7,8.1,8.5,8.9,9.3,9.9,10.2,...,12.7,13.1,13.5,14.2,14.8,15.5,16.4,17.1,18.0,19.0
1,서울특별시,5.4,5.7,6.0,6.4,6.8,7.2,7.7,8.3,8.7,...,12.0,12.6,13.0,13.8,14.4,15.2,16.1,16.8,17.6,18.5
2,부산광역시,6.0,6.4,6.8,7.3,7.8,8.3,8.9,9.6,10.2,...,14.0,14.6,15.3,16.3,17.1,18.2,19.4,20.4,21.5,22.6
3,대구광역시,5.9,6.2,6.6,6.9,7.4,7.8,8.3,8.9,9.3,...,12.2,12.7,13.2,14.0,14.7,15.6,16.6,17.5,18.4,19.6
4,인천광역시,5.4,5.7,5.9,6.3,6.6,6.9,7.2,7.7,8.0,...,10.3,10.7,11.0,11.7,12.3,13.0,14.0,14.8,15.6,16.6
5,광주광역시,5.6,5.8,6.1,6.4,6.8,7.1,7.5,8.1,8.3,...,10.8,11.3,11.7,12.4,12.8,13.4,14.2,14.8,15.6,16.5
6,대전광역시,5.5,5.7,6.0,6.3,6.6,6.9,7.2,7.7,8.1,...,10.3,10.9,11.3,12.0,12.7,13.5,14.4,15.2,16.1,17.0
7,울산광역시,4.0,4.2,4.4,4.7,5.0,5.3,5.6,6.0,6.3,...,8.3,8.8,9.3,10.0,10.7,11.5,12.6,13.6,14.7,15.9
8,세종특별자치시,0,0,0,0,0,0,0,0,0,...,12.7,10.6,10.0,9.6,9.3,9.4,9.8,10.1,10.5,11.0
9,경기도,5.7,5.9,6.2,6.5,6.8,7.1,7.4,7.8,8.1,...,10.2,10.5,10.8,11.4,11.9,12.5,13.2,13.9,14.7,15.6


In [6]:
old.rename(columns={'행정구역별(1)': 'Year'}, inplace=True)
old = old.set_index('Year')
columns_except_item = old.columns.difference(['Year'])
old[columns_except_item] = old[columns_except_item].apply(pd.to_numeric, errors='coerce')
pop_aging_rate = old.transpose()
pop_aging_rate

Year,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원특별자치도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
2000,7.0,5.4,6.0,5.9,5.4,5.6,5.5,4.0,0.0,5.7,9.3,9.1,11.2,10.3,11.9,10.7,8.4,8.0
2001,7.4,5.7,6.4,6.2,5.7,5.8,5.7,4.2,0.0,5.9,9.8,9.5,11.8,10.7,12.6,11.2,8.8,8.3
2002,7.7,6.0,6.8,6.6,5.9,6.1,6.0,4.4,0.0,6.2,10.3,10.0,12.3,11.3,13.3,11.7,9.1,8.7
2003,8.1,6.4,7.3,6.9,6.3,6.4,6.3,4.7,0.0,6.5,10.9,10.4,12.8,11.8,14.1,12.3,9.5,9.2
2004,8.5,6.8,7.8,7.4,6.6,6.8,6.6,5.0,0.0,6.8,11.5,10.9,13.1,12.5,14.9,12.9,9.9,9.6
2005,8.9,7.2,8.3,7.8,6.9,7.1,6.9,5.3,0.0,7.1,12.1,11.3,13.3,12.9,15.6,13.4,10.2,10.0
2006,9.3,7.7,8.9,8.3,7.2,7.5,7.2,5.6,0.0,7.4,12.8,11.8,13.7,13.5,16.3,13.8,10.7,10.5
2007,9.9,8.3,9.6,8.9,7.7,8.1,7.7,6.0,0.0,7.8,13.4,12.4,14.3,14.3,17.2,14.6,11.1,11.0
2008,10.2,8.7,10.2,9.3,8.0,8.3,8.1,6.3,0.0,8.1,13.9,12.7,14.5,14.7,17.6,15.1,11.4,11.5
2009,10.6,9.2,10.8,9.7,8.3,8.7,8.4,6.6,0.0,8.4,14.4,13.0,14.8,15.0,18.0,15.4,11.6,11.9


In [7]:
old_year = ['2000', '2001', '2002', '2003','2004','2005', '2006', '2007', '2008','2009','2010', '2011', '2012', '2013','2014','2015', '2016', '2017', '2018','2019','2020', '2021', '2022', '2023']
pop_aging_rate.insert(0, 'Year', old_year)
pop_aging_rate

Year,Year,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원특별자치도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
2000,2000,7.0,5.4,6.0,5.9,5.4,5.6,5.5,4.0,0.0,5.7,9.3,9.1,11.2,10.3,11.9,10.7,8.4,8.0
2001,2001,7.4,5.7,6.4,6.2,5.7,5.8,5.7,4.2,0.0,5.9,9.8,9.5,11.8,10.7,12.6,11.2,8.8,8.3
2002,2002,7.7,6.0,6.8,6.6,5.9,6.1,6.0,4.4,0.0,6.2,10.3,10.0,12.3,11.3,13.3,11.7,9.1,8.7
2003,2003,8.1,6.4,7.3,6.9,6.3,6.4,6.3,4.7,0.0,6.5,10.9,10.4,12.8,11.8,14.1,12.3,9.5,9.2
2004,2004,8.5,6.8,7.8,7.4,6.6,6.8,6.6,5.0,0.0,6.8,11.5,10.9,13.1,12.5,14.9,12.9,9.9,9.6
2005,2005,8.9,7.2,8.3,7.8,6.9,7.1,6.9,5.3,0.0,7.1,12.1,11.3,13.3,12.9,15.6,13.4,10.2,10.0
2006,2006,9.3,7.7,8.9,8.3,7.2,7.5,7.2,5.6,0.0,7.4,12.8,11.8,13.7,13.5,16.3,13.8,10.7,10.5
2007,2007,9.9,8.3,9.6,8.9,7.7,8.1,7.7,6.0,0.0,7.8,13.4,12.4,14.3,14.3,17.2,14.6,11.1,11.0
2008,2008,10.2,8.7,10.2,9.3,8.0,8.3,8.1,6.3,0.0,8.1,13.9,12.7,14.5,14.7,17.6,15.1,11.4,11.5
2009,2009,10.6,9.2,10.8,9.7,8.3,8.7,8.4,6.6,0.0,8.4,14.4,13.0,14.8,15.0,18.0,15.4,11.6,11.9


* 농가인구수 통계 데이터

In [8]:
pop_con = pd.read_csv("../data2/농가인구.csv")
con = pop_con.iloc[1:]
con = con.replace('-', 0).fillna(0)
con.rename(columns={'행정구역별(1)': 'Year'}, inplace=True)
con = con.set_index('Year')
con = con[con.columns[1:]].astype(int)
pop_contry = con.transpose()
pop_contry

Year,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
2011,2962113,11768,23565,50876,36196,35232,29332,33154,0,412351,183273,207174,378094,259552,379767,471076,336641,114062
2012,2911540,9927,22859,51514,34760,32903,28760,32523,0,407109,180144,202735,367944,263112,370614,465408,327931,113298
2013,2847435,9164,20486,50618,32368,32231,28816,32114,0,390378,177227,197410,356158,258880,370954,458791,320095,111745
2014,2751792,8689,19935,49187,32089,32062,27590,29687,15312,374872,172026,187752,328993,250581,355601,446920,310985,109510
2015,2569387,11555,20944,44542,32454,30746,25569,31228,16335,350200,175567,178248,308455,227431,318848,410141,293720,93404
2016,2496406,9599,20554,43743,31226,29800,24525,31278,14982,324562,171363,180565,296801,224661,321395,402247,280720,88385
2017,2422256,9374,19133,42395,29991,26761,23272,30577,14821,316573,160871,173436,288800,214548,315035,395275,274930,86463
2018,2314982,8605,17400,39903,27642,25347,21985,28859,14268,296350,154098,164387,276426,208600,306365,376562,265433,82751
2019,2244783,7586,16578,39239,27096,24101,20697,27208,13587,282343,148545,162083,263124,204124,297890,368693,258754,83133
2020,2314064,21897,28529,60630,31828,39873,32804,36806,17063,308677,151326,147904,266509,199157,280060,351375,259829,79797


In [9]:
con_year = ['2011', '2012', '2013','2014','2015', '2016', '2017', '2018','2019','2020', '2021', '2022']
pop_contry.insert(0, 'Year', con_year)
pop_contry

Year,Year,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
2011,2011,2962113,11768,23565,50876,36196,35232,29332,33154,0,412351,183273,207174,378094,259552,379767,471076,336641,114062
2012,2012,2911540,9927,22859,51514,34760,32903,28760,32523,0,407109,180144,202735,367944,263112,370614,465408,327931,113298
2013,2013,2847435,9164,20486,50618,32368,32231,28816,32114,0,390378,177227,197410,356158,258880,370954,458791,320095,111745
2014,2014,2751792,8689,19935,49187,32089,32062,27590,29687,15312,374872,172026,187752,328993,250581,355601,446920,310985,109510
2015,2015,2569387,11555,20944,44542,32454,30746,25569,31228,16335,350200,175567,178248,308455,227431,318848,410141,293720,93404
2016,2016,2496406,9599,20554,43743,31226,29800,24525,31278,14982,324562,171363,180565,296801,224661,321395,402247,280720,88385
2017,2017,2422256,9374,19133,42395,29991,26761,23272,30577,14821,316573,160871,173436,288800,214548,315035,395275,274930,86463
2018,2018,2314982,8605,17400,39903,27642,25347,21985,28859,14268,296350,154098,164387,276426,208600,306365,376562,265433,82751
2019,2019,2244783,7586,16578,39239,27096,24101,20697,27208,13587,282343,148545,162083,263124,204124,297890,368693,258754,83133
2020,2020,2314064,21897,28529,60630,31828,39873,32804,36806,17063,308677,151326,147904,266509,199157,280060,351375,259829,79797


* 농가인구의 고령화 통계 데이터

In [10]:
all = pd.read_excel("../eda_data/농가인구_고령화_통계.xlsx")
all

,Unnamed: 0,농가수,- 총가구 중 비중,- 농가당 가구원수,농가인구,- 총인구 중 비중,- 65세 이상 비중
0,2013,"1,142",6.2,2.5,"2,847",5.6,37.3
1,2014,"1,121",6.0,2.5,"2,752",5.4,39.1
2,2015,"1,089",5.6,2.4,"2,569",5.0,38.4
3,2016,"1,068",5.4,2.3,"2,496",4.9,40.3
4,2017,"1,042",5.2,2.3,"2,422",4.7,42.5
5,2018,"1,021",5.0,2.3,"2,315",4.5,44.7
6,2019,"1,007",4.8,2.2,"2,245",4.3,46.6
7,2020,"1,035",4.8,2.2,"2,314",4.5,42.3
8,2021,"1,031",4.7,2.2,"2,215",4.3,46.8
9,2022,"1,023",4.7,2.1,"2,166",4.2,49.8


In [11]:
all.rename(columns={all.columns[0]:"년도",
                          all.columns[1]:"농가수",
                          all.columns[2]:"총가구 중 비중",
                          all.columns[3]:"농가당 가구원 수",
                          all.columns[4]:"농가인구",
                          all.columns[5]:"총인구 중 비중",
                          all.columns[6]:"65세 이상 비중",},
                          inplace=True)

In [12]:
con_aging = all.replace('-', 0).fillna(0)
con_aging['농가인구'] = con_aging.농가인구.str.replace(',','')
con_aging['농가수'] = con_aging.농가수.str.replace(',','')
con_aging

,년도,농가수,총가구 중 비중,농가당 가구원 수,농가인구,총인구 중 비중,65세 이상 비중
0,2013,1142,6.2,2.5,2847,5.6,37.3
1,2014,1121,6.0,2.5,2752,5.4,39.1
2,2015,1089,5.6,2.4,2569,5.0,38.4
3,2016,1068,5.4,2.3,2496,4.9,40.3
4,2017,1042,5.2,2.3,2422,4.7,42.5
5,2018,1021,5.0,2.3,2315,4.5,44.7
6,2019,1007,4.8,2.2,2245,4.3,46.6
7,2020,1035,4.8,2.2,2314,4.5,42.3
8,2021,1031,4.7,2.2,2215,4.3,46.8
9,2022,1023,4.7,2.1,2166,4.2,49.8


In [13]:
con_aging['농가수'] = con_aging['농가수'].astype(int)
con_aging['총가구 중 비중'] = con_aging['총가구 중 비중'].astype(float)
con_aging['농가당 가구원 수'] = con_aging['농가당 가구원 수'].astype(float)
con_aging['농가인구'] = con_aging['농가인구'].astype(int)
con_aging['총인구 중 비중'] = con_aging['총인구 중 비중'].astype(float)
con_aging['65세 이상 비중'] = con_aging['65세 이상 비중'].astype(float)
con_aging

,년도,농가수,총가구 중 비중,농가당 가구원 수,농가인구,총인구 중 비중,65세 이상 비중
0,2013,1142,6.2,2.5,2847,5.6,37.3
1,2014,1121,6.0,2.5,2752,5.4,39.1
2,2015,1089,5.6,2.4,2569,5.0,38.4
3,2016,1068,5.4,2.3,2496,4.9,40.3
4,2017,1042,5.2,2.3,2422,4.7,42.5
5,2018,1021,5.0,2.3,2315,4.5,44.7
6,2019,1007,4.8,2.2,2245,4.3,46.6
7,2020,1035,4.8,2.2,2314,4.5,42.3
8,2021,1031,4.7,2.2,2215,4.3,46.8
9,2022,1023,4.7,2.1,2166,4.2,49.8


* 시도별 인구밀도 데이터 + 만약 넣는다면 도시화율 관련도 넣고, 분석

In [14]:
df_pop_gd = pd.read_csv("../eda_data/인구밀도_인구주택조사기준.csv")
df_pop_gd = df_pop_gd.replace('-', 0).fillna(0)
df_pop_gd = df_pop_gd[df_pop_gd.columns[0:]].astype(float)
df_pop_gd

,시점,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
0,2000.0,463.9,16342.2,4820.4,2801.0,2566.2,2697.8,2534.5,960.4,0.0,886.4,89.7,197.3,214.9,234.9,166.6,143.2,283.2,278.0
1,2005.0,474.5,16221.0,4609.4,2786.5,2546.3,2827.5,2673.0,992.5,0.0,1028.1,88.2,196.5,219.7,221.5,150.7,137.1,290.5,287.8
2,2010.0,485.6,16188.9,4452.3,2767.4,2587.5,2945.6,2781.2,1022.3,0.0,1119.3,88.2,203.4,235.0,220.3,142.2,136.6,300.0,287.7
3,2015.0,509.2,16364.0,4479.9,2791.0,2755.5,2998.8,2852.3,1099.6,439.0,1226.4,90.2,214.6,256.6,227.4,146.1,140.8,316.4,327.5
4,2016.0,511.0,16202.2,4468.8,2785.3,2741.4,2995.7,2846.9,1099.2,521.7,1244.4,90.4,216.5,259.2,227.2,145.8,140.9,316.9,337.1
5,2017.0,512.4,16095.8,4437.4,2776.2,2752.3,2985.3,2828.5,1090.4,595.0,1261.6,90.4,217.5,262.9,226.3,145.3,140.7,317.4,346.9
6,2018.0,514.4,15983.7,4409.8,2766.7,2761.4,2973.2,2801.0,1083.4,671.9,1286.2,90.3,218.8,265.1,225.3,145.0,140.4,317.9,355.8
7,2019.0,515.7,15926.9,4379.7,2750.3,2776.6,2972.7,2777.5,1076.9,727.3,1305.0,90.3,220.0,265.4,224.0,144.8,140.2,317.6,359.4
8,2020.0,516.2,15839.0,4348.9,2728.6,2765.1,2948.5,2758.1,1069.0,761.3,1325.3,90.4,220.3,264.0,223.4,144.9,138.9,316.2,362.6
9,2021.0,515.2,15650.1,4316.4,2702.2,2772.8,2944.0,2742.8,1055.0,787.7,1338.9,90.4,219.4,263.8,221.4,143.9,138.4,313.6,363.8


* 1차산업 숙련 종사자 통계

In [15]:
pop_work = pd.read_csv("../data2/직업별_취업자.csv")
pop_work = pop_work[pop_work.columns[0:]].astype(int)
pop_work.rename(columns={'6 농림어업 숙련종사자': '1차산업_숙련_종사자'}, inplace=True)
pop_work

,시점,1차산업_숙련_종사자
0,2004,1775
1,2005,1710
2,2006,1666
3,2007,1617
4,2008,1579
5,2009,1535
6,2010,1460
7,2011,1439
8,2012,1427
9,2013,1420


* 시도별, 연령별, 귀농인수 변화 통계

In [16]:
df_pop_home = pd.read_csv("../eda_data/시도_연령별_귀농인수.csv")
df_pop_home = df_pop_home.replace('X', 0).fillna(0)
df_pop_home.rename(columns={'행정구역별': '시도별', '귀농인수 (명)' : '귀농인수'}, inplace=True)
df_pop_home['귀농인수'] = df_pop_home['귀농인수'].astype(int)
df_pop_home

,시점,시도별,연령별,귀농인수
0,2013,전국,30대이하,1174
1,2013,전국,40대,2348
2,2013,전국,50대,4077
3,2013,전국,60대,2186
4,2013,전국,70대이상,527
...,...,...,...,...
745,2022,제주특별자치도,30대이하,38
746,2022,제주특별자치도,40대,61
747,2022,제주특별자치도,50대,82
748,2022,제주특별자치도,60대,59


* 시도별 귀농인구수 변화 통계

In [22]:
pop_go_con = pd.read_csv("../eda_data/시도별_귀농인구수.csv")
pop_go_con = pop_go_con.replace('X', 0).fillna(0)
pop_go_con = pop_go_con.drop(0, axis=0)
pop_go_con = pop_go_con.drop('연령별', axis=1)
pop_go_con.reset_index(drop=True, inplace=True)
pop_go_con

,시점,전국,부산광역시,대구광역시,인천광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
0,2013,10312,22,56,82,69,73,882,845,835,1123,1140,1689,1957,1289,250
1,2014,10904,30,70,88,82,69,898,809,897,1226,1170,1773,2134,1361,297
2,2015,12114,31,72,104,84,64,1074,1006,940,1390,1181,1894,2249,1634,391
3,2016,13019,27,92,125,88,84,1298,1070,1063,1438,1273,1937,2352,1661,511
4,2017,12763,33,89,138,90,72,1182,1090,959,1405,1374,1938,2338,1686,369
5,2018,12055,25,59,134,85,88,987,1055,925,1328,1335,2039,2196,1518,281
6,2019,11504,18,76,146,67,71,999,948,847,1268,1327,2020,2156,1323,238
7,2020,12570,35,58,122,78,71,1118,947,938,1502,1511,2358,2252,1349,231
8,2021,14461,26,71,171,108,90,1288,1022,1086,1821,1524,2579,2726,1699,250
9,2022,12660,24,62,118,75,81,1207,954,962,1595,1237,1987,2579,1530,249


In [23]:
pop_go_con = pop_go_con[pop_go_con.columns[0:]].astype(int)
pop_go_con

,시점,전국,부산광역시,대구광역시,인천광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
0,2013,10312,22,56,82,69,73,882,845,835,1123,1140,1689,1957,1289,250
1,2014,10904,30,70,88,82,69,898,809,897,1226,1170,1773,2134,1361,297
2,2015,12114,31,72,104,84,64,1074,1006,940,1390,1181,1894,2249,1634,391
3,2016,13019,27,92,125,88,84,1298,1070,1063,1438,1273,1937,2352,1661,511
4,2017,12763,33,89,138,90,72,1182,1090,959,1405,1374,1938,2338,1686,369
5,2018,12055,25,59,134,85,88,987,1055,925,1328,1335,2039,2196,1518,281
6,2019,11504,18,76,146,67,71,999,948,847,1268,1327,2020,2156,1323,238
7,2020,12570,35,58,122,78,71,1118,947,938,1502,1511,2358,2252,1349,231
8,2021,14461,26,71,171,108,90,1288,1022,1086,1821,1524,2579,2726,1699,250
9,2022,12660,24,62,118,75,81,1207,954,962,1595,1237,1987,2579,1530,249


### 2. 가공한 데이터를 공동 DB에 table로 저장하기

<hr>

* 연령별 전국민 수 통계 데이터를 sql에 table로 넣기

In [9]:
remote = mysql.connector.connect(
    host = "database-1.cdv9nppqt3kw.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "****",
    password = "****",
    database = "AutoFarming"
)

In [10]:
son_cursor = remote.cursor()
son_cursor.execute('create table population_age (Year int, Age varchar(64), Age_range varchar(64), Number int)')

In [11]:
sql = '''insert into population_age values (%s, %s, %s, %s)'''
cursor = remote.cursor(buffered=True)

for i, row in pop_all.iterrows():
    cursor.execute(sql, tuple(row))
    print(tuple(row))
    remote.commit()

(1960, '0 - 4세', '0.영유아', 4590812)
(1960, '5 - 9세', '1.아동', 3350533)
(1960, '10 - 14세', '1.아동', 2646238)
(1960, '15 - 19세', '2.청소년', 2418444)
(1960, '20 - 24세', '3.청년', 2322705)
(1960, '25 - 29세', '3.청년', 1933080)
(1960, '30 - 34세', '4.중년', 1556328)
(1960, '35 - 39세', '4.중년', 1370125)
(1960, '40 - 44세', '4.중년', 1125034)
(1960, '45 - 49세', '4.중년', 979930)
(1960, '50 - 54세', '5.장년', 845256)
(1960, '55 - 59세', '5.장년', 628974)
(1960, '60 - 64세', '5.장년', 518465)
(1960, '65 - 69세', '6.노년', 345675)
(1960, '70 - 74세', '6.노년', 210394)
(1960, '75 - 79세', '6.노년', 111199)
(1960, '80세이상', '6.노년', 59182)
(1970, '0 - 4세', '0.영유아', 4548177)
(1970, '5 - 9세', '1.아동', 4629882)
(1970, '10 - 14세', '1.아동', 4531308)
(1970, '15 - 19세', '2.청소년', 3269356)
(1970, '20 - 24세', '3.청년', 2569044)
(1970, '25 - 29세', '3.청년', 2335969)
(1970, '30 - 34세', '4.중년', 2237747)
(1970, '35 - 39세', '4.중년', 1866982)
(1970, '40 - 44세', '4.중년', 1467129)
(1970, '45 - 49세', '4.중년', 1271554)
(1970, '50 - 54세', '5.장년', 1017150)
(1970, '

* 년도별 고령화비율 통계 데이터를 sql에 table로 넣기

In [ ]:
son_cursor = remote.cursor()
son_cursor.execute('create table pop_aging_rate (Year float, 전국 float, 서울특별시 float, 부산광역시 float, 대구광역시 float, 인천광역시 float, 광주광역시 float, 대전광역시 float, 울산광역시 float, 세종특별자치시 float, 경기도 float, 강원특별자치도 float, 충청북도 float, 충청남도 float, 전라북도 float, 전라남도 float, 경상북도 float, 경상남도 float, 제주특별자치도 float)')

In [ ]:
sql = '''insert into pop_aging_rate values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
cursor = remote.cursor(buffered=True)

for i, row in pop_aging_rate.iterrows():
    cursor.execute(sql, tuple(row))
    print(tuple(row))
    remote.commit()

('2000', 7.0, 5.4, 6.0, 5.9, 5.4, 5.6, 5.5, 4.0, 0.0, 5.7, 9.3, 9.1, 11.2, 10.3, 11.9, 10.7, 8.4, 8.0)
('2001', 7.4, 5.7, 6.4, 6.2, 5.7, 5.8, 5.7, 4.2, 0.0, 5.9, 9.8, 9.5, 11.8, 10.7, 12.6, 11.2, 8.8, 8.3)
('2002', 7.7, 6.0, 6.8, 6.6, 5.9, 6.1, 6.0, 4.4, 0.0, 6.2, 10.3, 10.0, 12.3, 11.3, 13.3, 11.7, 9.1, 8.7)
('2003', 8.1, 6.4, 7.3, 6.9, 6.3, 6.4, 6.3, 4.7, 0.0, 6.5, 10.9, 10.4, 12.8, 11.8, 14.1, 12.3, 9.5, 9.2)
('2004', 8.5, 6.8, 7.8, 7.4, 6.6, 6.8, 6.6, 5.0, 0.0, 6.8, 11.5, 10.9, 13.1, 12.5, 14.9, 12.9, 9.9, 9.6)
('2005', 8.9, 7.2, 8.3, 7.8, 6.9, 7.1, 6.9, 5.3, 0.0, 7.1, 12.1, 11.3, 13.3, 12.9, 15.6, 13.4, 10.2, 10.0)
('2006', 9.3, 7.7, 8.9, 8.3, 7.2, 7.5, 7.2, 5.6, 0.0, 7.4, 12.8, 11.8, 13.7, 13.5, 16.3, 13.8, 10.7, 10.5)
('2007', 9.9, 8.3, 9.6, 8.9, 7.7, 8.1, 7.7, 6.0, 0.0, 7.8, 13.4, 12.4, 14.3, 14.3, 17.2, 14.6, 11.1, 11.0)
('2008', 10.2, 8.7, 10.2, 9.3, 8.0, 8.3, 8.1, 6.3, 0.0, 8.1, 13.9, 12.7, 14.5, 14.7, 17.6, 15.1, 11.4, 11.5)
('2009', 10.6, 9.2, 10.8, 9.7, 8.3, 8.7, 8.4, 6.6

* 농가인구수 통계 데이터를 sql에 table로 넣기

In [ ]:
son_cursor = remote.cursor()
son_cursor.execute('create table pop_contry (Year float, 전국 float, 서울특별시 float, 부산광역시 float, 대구광역시 float, 인천광역시 float, 광주광역시 float, 대전광역시 float, 울산광역시 float, 세종특별자치시 float, 경기도 float, 강원특별자치도 float, 충청북도 float, 충청남도 float, 전라북도 float, 전라남도 float, 경상북도 float, 경상남도 float, 제주특별자치도 float)')

In [ ]:
sql = '''insert into pop_contry values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
cursor = remote.cursor(buffered=True)

for i, row in pop_contry.iterrows():
    cursor.execute(sql, tuple(row))
    print(tuple(row))
    remote.commit()

('2011', 2962113, 11768, 23565, 50876, 36196, 35232, 29332, 33154, 0, 412351, 183273, 207174, 378094, 259552, 379767, 471076, 336641, 114062)
('2012', 2911540, 9927, 22859, 51514, 34760, 32903, 28760, 32523, 0, 407109, 180144, 202735, 367944, 263112, 370614, 465408, 327931, 113298)
('2013', 2847435, 9164, 20486, 50618, 32368, 32231, 28816, 32114, 0, 390378, 177227, 197410, 356158, 258880, 370954, 458791, 320095, 111745)
('2014', 2751792, 8689, 19935, 49187, 32089, 32062, 27590, 29687, 15312, 374872, 172026, 187752, 328993, 250581, 355601, 446920, 310985, 109510)
('2015', 2569387, 11555, 20944, 44542, 32454, 30746, 25569, 31228, 16335, 350200, 175567, 178248, 308455, 227431, 318848, 410141, 293720, 93404)
('2016', 2496406, 9599, 20554, 43743, 31226, 29800, 24525, 31278, 14982, 324562, 171363, 180565, 296801, 224661, 321395, 402247, 280720, 88385)
('2017', 2422256, 9374, 19133, 42395, 29991, 26761, 23272, 30577, 14821, 316573, 160871, 173436, 288800, 214548, 315035, 395275, 274930, 86463

* 농가인구의 고령화 통계 데이터를 sql에 table로 넣기

In [ ]:
son_cursor = remote.cursor()
son_cursor.execute('create table con_aging (Year int, 농가수 int, 총가구_중_비중 float, 농가당_가구원_수 float, 농가인구 int, 총인구_중_비중 float, 65세_이상_비중 float)')

In [ ]:
sql = '''insert into con_aging values (%s, %s, %s, %s, %s, %s, %s)'''
cursor = remote.cursor(buffered=True)

for i, row in con_aging.iterrows():
    cursor.execute(sql, tuple(row))
    print(tuple(row))
    remote.commit()

(2013.0, 1142.0, 6.2, 2.5, 2847.0, 5.6, 37.3)
(2014.0, 1121.0, 6.0, 2.5, 2752.0, 5.4, 39.1)
(2015.0, 1089.0, 5.6, 2.4, 2569.0, 5.0, 38.4)
(2016.0, 1068.0, 5.4, 2.3, 2496.0, 4.9, 40.3)
(2017.0, 1042.0, 5.2, 2.3, 2422.0, 4.7, 42.5)
(2018.0, 1021.0, 5.0, 2.3, 2315.0, 4.5, 44.7)
(2019.0, 1007.0, 4.8, 2.2, 2245.0, 4.3, 46.6)
(2020.0, 1035.0, 4.8, 2.2, 2314.0, 4.5, 42.3)
(2021.0, 1031.0, 4.7, 2.2, 2215.0, 4.3, 46.8)
(2022.0, 1023.0, 4.7, 2.1, 2166.0, 4.2, 49.8)


* 1차 산업 숙련 종사자 데이터를 sql에 table로 넣기

In [ ]:
son_cursor = remote.cursor()
son_cursor.execute('create table pop_work (Year int, 1차산업_숙련_종사자 int)')

In [ ]:
sql = '''insert into pop_work values (%s, %s)'''
cursor = remote.cursor(buffered=True)

for i, row in pop_work.iterrows():
    cursor.execute(sql, tuple(row))
    print(tuple(row))
    remote.commit()

(2004, 1775)
(2005, 1710)
(2006, 1666)
(2007, 1617)
(2008, 1579)
(2009, 1535)
(2010, 1460)
(2011, 1439)
(2012, 1427)
(2013, 1420)
(2014, 1357)
(2015, 1243)
(2016, 1186)
(2017, 1198)


* 시도별 귀농인구수 통계 데이터를 sql에 table로 넣기

In [26]:
son_cursor = remote.cursor()
son_cursor.execute('create table pop_go_con (Year int, 전국 int, 부산광역시 int, 대구광역시 int, 인천광역시 int, 울산광역시 int, 세종특별자치시 int, 경기도 int, 강원도 int, 충청북도 int, 충청남도 int, 전라북도 int, 전라남도 int, 경상북도 int, 경상남도 int, 제주특별자치도 int)')

In [27]:
sql = '''insert into pop_go_con values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
cursor = remote.cursor(buffered=True)

for i, row in pop_go_con.iterrows():
    cursor.execute(sql, tuple(row))
    print(tuple(row))
    remote.commit()

(2013, 10312, 22, 56, 82, 69, 73, 882, 845, 835, 1123, 1140, 1689, 1957, 1289, 250)
(2014, 10904, 30, 70, 88, 82, 69, 898, 809, 897, 1226, 1170, 1773, 2134, 1361, 297)
(2015, 12114, 31, 72, 104, 84, 64, 1074, 1006, 940, 1390, 1181, 1894, 2249, 1634, 391)
(2016, 13019, 27, 92, 125, 88, 84, 1298, 1070, 1063, 1438, 1273, 1937, 2352, 1661, 511)
(2017, 12763, 33, 89, 138, 90, 72, 1182, 1090, 959, 1405, 1374, 1938, 2338, 1686, 369)
(2018, 12055, 25, 59, 134, 85, 88, 987, 1055, 925, 1328, 1335, 2039, 2196, 1518, 281)
(2019, 11504, 18, 76, 146, 67, 71, 999, 948, 847, 1268, 1327, 2020, 2156, 1323, 238)
(2020, 12570, 35, 58, 122, 78, 71, 1118, 947, 938, 1502, 1511, 2358, 2252, 1349, 231)
(2021, 14461, 26, 71, 171, 108, 90, 1288, 1022, 1086, 1821, 1524, 2579, 2726, 1699, 250)
(2022, 12660, 24, 62, 118, 75, 81, 1207, 954, 962, 1595, 1237, 1987, 2579, 1530, 249)


In [12]:
remote.close()